In [ ]:
from pythreejs.install import install
install(user=True, symlink=True)

In [ ]:
from ipywidgets import Controller, FloatSlider
pad = Controller()

In [ ]:
pad

# We can easily wire the gamepad buttons and axes to other widgets

In [ ]:
import contextlib
from traitlets.traitlets import _validate_link


class transform_link(object):
    """Link the trait of a source object with traits of target objects.

    Parameters
    ----------
    source : (object, attribute name) pair
    target : (object, attribute name) pair
    validation: callable

    Examples
    --------

    >>> c = transform_link((src, 'value'), (tgt, 'value'))
    >>> src.value = 5  # updates target objects
    >>> tgt.value = 6  # does not update source object
    """
    updating = False

    def __init__(self, source, target, validation):
        self._validate = validation
        _validate_link(source, target)
        self.source, self.target = source, target
        try:
            setattr(target[0], target[1],
                    self._validate(getattr(source[0], source[1])))
        finally:
            self.source[0].on_trait_change(self._update, self.source[1])

    @contextlib.contextmanager
    def _busy_updating(self):
        self.updating = True
        try:
            yield
        finally:
            self.updating = False

    def _update(self, name, old, new):
        if self.updating:
            return
        with self._busy_updating():
            setattr(self.target[0], self.target[1], self._validate(new))

    def unlink(self):
        self.source[0].on_trait_change(self._update, self.source[1], remove=True)
        self.source, self.target = None, None

In [ ]:
slider = FloatSlider(min=-1, max=1, description='Axis 0')

def affine(constant, factor):
    return lambda x: constant + factor * x

pad.links = []

def setup():
    if pad.connected:
        pad.links.append(transform_link((pad.axes[0], 'value'),
                                        (slider, 'value'),
      affine(0.5 * (slider.max + slider.min), 0.5 * (slider.max - slider.min))))
    if not pad.connected:
        for l in pad.links:
            l.unlink()
        pad.links = []
        
pad.on_trait_change(setup, name='connected')
setup()

In [ ]:
from ipywidgets import FloatSlider

In [ ]:
slider

In [ ]:
from traitlets import link
from ipywidgets import HTML, VBox
import numpy as np
from pythreejs import *

nx,ny=(20,20)
xmax=1
x = np.linspace(-xmax,xmax,nx)
y = np.linspace(-xmax,xmax,ny)
xx, yy = np.meshgrid(x,y)
z = xx**2-yy**2
#z[6,1] = float('nan')
surf_g = SurfaceGeometry(z=list(z[::-1].flat), 
                          width=2*xmax,
                          height=2*xmax,
                          width_segments=nx-1,
                          height_segments=ny-1)

surf = Mesh(geometry=surf_g, material=LambertMaterial(map=height_texture(z[::-1], 'YlGnBu_r')))
surfgrid = SurfaceGrid(geometry=surf_g, material=LineBasicMaterial(color='black'))
hover_point = Mesh(geometry=SphereGeometry(radius=0.05), material=LambertMaterial(color='hotpink'))
scene = Scene(children=[surf, surfgrid, hover_point, AmbientLight(color=0x777777)])
c = PerspectiveCamera(position=[0,3,3], up=[0,0,1], 
                      children=[DirectionalLight(color='white', position=[3,5,1], intensity=0.6)])
click_picker = Picker(root=surf, event='dblclick')
hover_picker = Picker(root=surf, event='mousemove')
renderer = Renderer(camera=c, scene = scene, controls=[OrbitControls(controlling=c), click_picker, hover_picker])

def f(name, value):
    print "Clicked on %s"%value
    point = Mesh(geometry=SphereGeometry(radius=0.05), 
                              material=LambertMaterial(color='red'),
                             position=value)
    scene.children = list(scene.children)+[point]
click_picker.on_trait_change(f, 'point')

link((hover_point, 'position'), (hover_picker, 'point'))

h = HTML()
def g(name, value):
    h.value="Green point at (%.3f, %.3f, %.3f)"%tuple(value)
g(None, hover_point.position)
hover_picker.on_trait_change(g, 'point')
VBox([h, renderer])

# Terrain modeling? Let us check out the Grand Canyon

In [ ]:
from pythreejs import *
import numpy as np
import gdal as gd

In [ ]:
gc_ds = gd.Open('gc_dem.tif')
dem = gc_ds.ReadAsArray()[::10, ::10]
gt = gc_ds.GetGeoTransform()#

z = (dem - np.mean(dem) ) / 1000
nx, ny = z.shape

surf_g = SurfaceGeometry(z=list(z.flat), height_segments=nx - 1, width_segments=ny - 1)
surf = Mesh(geometry=surf_g, material=LambertMaterial(map=height_texture(z, colormap='terrain')),
            scale=(10, 10, 1))
scene = Scene(children=[AmbientLight(color=0x777777),
                        surf, DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])

In [ ]:
c = PerspectiveCamera(position=[0, 10, 10], up=[0, 0, 1], 
                      children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)],
                      aspect=2)
width = 950
height = width / c.aspect
c.look_at(c.position, (1, 0, 0))
fly_controls = FlyControls(controlling=c)
renderer = Renderer(camera=c, scene=scene, width=width, height=height,
                    controls=[fly_controls])

In [ ]:
c.position = [0, 10, 10]
c.look_at(c.position, (1, 0, 0))

In [ ]:
renderer

# Let us controll the camera position with the gamepad

In [ ]:
factor = 10
def affine(constant, factor):
    return lambda x: constant + factor * x

pad.links = []

def setup():
    if pad.connected:
        pad.links.append(transform_link((pad.axes[1], 'value'),
                                        (fly_controls, 'pitch'),
                         affine(0.0, factor)))
        pad.links.append(transform_link((pad.axes[0], 'value'),
                                        (fly_controls, 'roll'),
                         affine(0.0, -factor)))
        pad.links.append(transform_link((pad.axes[3], 'value'),
                                        (fly_controls, 'forward_speed'),
                         affine(0.0, 2 * factor)))
        pad.links.append(transform_link((pad.axes[2], 'value'),
                                        (fly_controls, 'yaw'),
                         affine(0.0, factor)))
        pad.links.append(transform_link((pad.buttons[5], 'value'),
                                        (surf, 'scale'),
                         lambda x: (10, 10, 1 - x)))
    if not pad.connected:
        for l in pad.links:
            l.unlink()
        pad.links = []
        
pad.on_trait_change(setup, name='connected')
setup()